In [10]:
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.set_printoptions(suppress=True)

%matplotlib inline

In [12]:
files = glob.glob('./dat/match*')

print(len(files), 'matches')

29478 matches


In [81]:
all_tiers = []
all_platforms = []
all_summonerNames = []
all_accountIDs = []
all_versions = []

for i, file in enumerate(files):
    if i % 1000 == 0: print i
    with open(file, 'r') as fp:
        match = json.load(fp)
        tier = []
        platform= []
        summonerName = []
        accountID = []
        try:
            tier = [match['participants'][i]['highestAchievedSeasonTier'] for i in range(10)]
            summonerName = [match['participantIdentities'][i]['player']['summonerName'] for i in range(10)]
            accountID = [match['participantIdentities'][i]['player']['accountId'] for i in range(10)]
            platform = [match['participantIdentities'][i]['player']['currentPlatformId'] for i in range(10)]
        except:
            continue
#                print(file) # indicates that this was PC
        cur_version = match['gameVersion']
        all_versions.extend([cur_version]*10)
        all_tiers.extend(tier)
        all_platforms.extend(platform)
        all_summonerNames.extend(summonerName)
        all_accountIDs.extend(accountID)

In [82]:
len(all_tiers)
print([len(x) for x in [all_tiers, all_summonerNames, all_accountIDs, all_versions]])

[288680, 288680, 288680, 288680]


In [83]:
np.unique(all_versions)

array(['4.10.0.379', '4.11.0.386', '4.11.0.399', '8.10.229.7328',
       '8.11.231.7304', '8.11.232.7066', '8.11.232.8721', '8.12.233.4186',
       '8.13.235.3406', '8.13.235.9749', '8.14.238.1713', '8.14.238.9731',
       '8.14.239.6309', '8.15.239.9021', '8.2.216.1199', '8.3.217.1022',
       '8.4.218.8787', '8.5.220.3006', '8.6.222.2149', '8.7.223.9264',
       '8.7.224.5563', '8.8.225.6906', '8.8.226.7254', '8.9.227.7511',
       '8.9.228.4283'], dtype='<U13')

In [85]:
version_number = '8'
version_numbers = [x[0] for x in all_versions]
valid_ind = np.where(np.array(version_numbers) == version_number)[0]
print(len(valid_ind), 'matches (by version)')

288320 matches (by version)


In [86]:
ranking = {
    'UNRANKED' : 0,
    'BRONZE' : 1,
    'SILVER' : 2,
    'GOLD' : 3,
    'PLATINUM' : 4,
    'DIAMOND' : 5,
    'MASTER' : 6,
    'CHALLENGER' : 7
}

# get rankings

rankings = {}

n = len(all_summonerNames)

for i in range(n):
    cur_summoner = all_summonerNames[i]
    if cur_summoner in rankings:
        rankings[cur_summoner].append(ranking[all_tiers[i]])
    else:
        rankings[cur_summoner] = [ranking[all_tiers[i]]]
        
# get max rankings

max_rankings = {}

for key, val in rankings.items():
    max_rankings[key] = max(val)

min_rankings = {}

for key, val in rankings.items():
    min_rankings[key] = min(val)

In [87]:
unique_names, u, counts = np.unique(all_summonerNames, return_index=True, return_counts=True)

indices = np.argsort(-1*counts)
unique_summoner_ids = unique_names[indices]
counts = counts[indices]
regions = np.array(all_platforms)[u][indices]
account_ids = np.array(all_accountIDs)[u][indices]

print(len(unique_names), 'accounts')

62604 accounts


In [107]:
valid_account_ids = []
valid_summoner_ids = []
for i, cur_summoner_id in enumerate(unique_summoner_ids):
    if max_rankings[cur_summoner_id] == 7 and min_rankings[cur_summoner_id] >= 6:
        if regions[i] == 'NA1':
            valid_account_ids.append(account_ids[i])
            valid_summoner_ids.append(cur_summoner_id)
            print(regions[i], counts[i], account_ids[i], cur_summoner_id)
            
print(len(valid_account_ids), 'challenger accounts')

NA1 532 50656739 Julien
NA1 508 32746443 Shiphtur
NA1 424 33689527 Morïarty
NA1 420 32850898 Descraton
NA1 398 32281250 Tony Top
NA1 387 32639237 Imaqtpie
NA1 310 40424646 Nightblue3
NA1 304 37560515 WARMACHINE HYPER
NA1 300 44508344 Piggy Kitten
NA1 297 33064075 BukZacH
NA1 275 208282883 mancloud2
NA1 244 37281196 Scrandor
NA1 237 36541962 Shout
NA1 232 42953504 Erry
NA1 230 203021036 røvex
NA1 226 35872324 Gweiss
NA1 224 34486403 dog2
NA1 221 233968031 Winter
NA1 219 36154943 Pants are Dragon
NA1 207 33773329 Day after day
NA1 204 33653150 An Obese Panda
NA1 196 44711577 Acoldblazeolive
NA1 193 182929 Yassuo
NA1 188 34159539 Teesum
NA1 176 200465862 insanityxxx
NA1 174 32921958 freelancer
NA1 171 43891192 A Good Sir
NA1 170 50589935 Kazahana
NA1 168 233871907 Liquid V1per
NA1 168 48340486 Tuesdayy
NA1 167 41866574 ULookSoCharming
NA1 166 50922751 AsianSGpotato
NA1 166 38188634 Titan Dweevil
NA1 160 45657551 Swkeeee
NA1 160 202988570 TF Blade
NA1 159 233033899 GGEA Neøø
NA1 155 365239

313 challenger accounts


In [109]:
valid_summoner_ids.reverse()
valid_account_ids.reverse()

In [110]:
for i, x in enumerate(valid_account_ids):
    print(x, valid_summoner_ids[i])

237028675 MLGB Clown
207581437 Leaves
39719059 Ivan Pavlov
227636156 Pick Winnie
41897392 Monsıeur
35177464 Manual Of Youth
35661831 Márko
38268014 Muckey
34361058 Blue Bird1
237257900 Bie da w pengyou
33064075 CC Zach
200950816 CC Dean
201195334 Cassiel
239623124 2001 07 02
201176848 Anegidin Nieb
228355982 Goon
235960023 HappyGames
48901673 Hachimitsu
241467084 Duo Abuse
32766 Dyrus
36564875 duoklng1
34389376 funnelcakefan69x
35177464 Yang Neng Jin
42591150 vii is a cutie
201176848 int is success
40424646 lIlIlIIlIlIl
212650893 i need some suga
214201438 no hh
229686398 mønkaS
201792142 SxYs921Klx
229692166 TakeNotezBro
35143315 Sessh
235114359 Spica 16
228337363 get laid
200499050 why talk
34721678 EnGeeeOh
237987641 nihonjin
34389376 saemi
207797332 DesoIatiôn      
23099233 llIllIllIlllIllI
36580110 OddOJ
36882691 Luqun
206113598 Terina
229692166 7 2 1993
223913076 Teamfighting
31757124 Hungry Like
207797332 300kg XIaoJieJie
35604587 LoopGoop
44275597 HIKASHIKUN
50892184 Marry Me


In [203]:
print(len(valid_account_ids))

313


In [197]:
from riotwatcher import RiotWatcher
import time

api_key = 'RGAPI-325783d1-b9e4-4c06-8e3b-64334bd4ea97'

watcher = RiotWatcher(api_key)

my_region = 'na1'

from progressbar import AnimatedMarker, Bar, BouncingBar, Counter, ETA, \
    AdaptiveETA, FileTransferSpeed, FormatLabel, Percentage, \
    ProgressBar, ReverseBar, RotatingMarker, \
    SimpleProgress, Timer, UnknownLength

def show_progress_bar(display_string, total_duration, num_increments):
    widgets = [display_string, Percentage(), ' ', Bar(),
               ' ', ETA()]
    pbar = ProgressBar(widgets=widgets, maxval=num_increments).start()
    time_increment = total_duration / num_increments
    s1 = time.time()
    for i in range(num_increments):
        time.sleep(time_increment)
        pbar.update(i+1)
    pbar.finish()
    e1 = time.time()
    print(e1-s1, 'seconds elapsed')
display_string = 'Waiting for API timer to reset: '

In [206]:
n = len(valid_account_ids)
api_call_counter = 0
all_match_ids = []
num_matches = []

file_accounts = './challenger_accounts.txt'
file_matches = './challenger_matches.txt'

with open(file_accounts, 'a') as fa:
    with open(file_matches, 'a') as fb:
        for i in range(14,n):
            cur_begin_index = 0
            found_invalid = 0
            try_account = True
            account_match_ids = []
            while(try_account):
                if api_call_counter == 100:
                    display_string = 'i = %d: Waiting for API timer to reset: ' % (i)
                    show_progress_bar(display_string, total_duration=120, num_increments=100)
                    api_call_counter = 0
                try:
                    api_call_counter += 1
                    matches = watcher.match.matchlist_by_account(my_region,
                                                             account_id = valid_account_ids[i],
                                                             queue = [400, 420, 430, 440],
                                                             begin_index = cur_begin_index,
                                                             season = 11)
                except:
                    print("Error with %d %s" % (valid_account_ids[i], valid_summoner_ids[i]))
                    found_invalid = 1
                    break
                if len(matches['matches']) < 100:
                    try_account = False
                cur_begin_index += 100
                match_ids = [x['gameId'] for x in matches['matches']]
                account_match_ids.extend(match_ids)
            if found_invalid == 1:
                continue
            num_matches.append(len(account_match_ids))
            account_str = ('{"account_id": "%d", "summoner_id": "%s", "num_matches": "%d"},\n' %
                   (valid_account_ids[i], valid_summoner_ids[i], len(account_match_ids))
                  )
            print(account_str[:-1])
            match_ids_as_string = '\n'.join([str(x) for x in account_match_ids]) + '\n'
            fa.write(account_str)
            fb.write(match_ids_as_string)
            all_match_ids.extend(account_match_ids)

Error with 201176848 Anegidin Nieb
{"account_id": "228355982", "summoner_id": "Goon", "num_matches": "9"},
{"account_id": "235960023", "summoner_id": "HappyGames", "num_matches": "249"},
{"account_id": "48901673", "summoner_id": "Hachimitsu", "num_matches": "570"},
{"account_id": "241467084", "summoner_id": "Duo Abuse", "num_matches": "96"},
{"account_id": "32766", "summoner_id": "Dyrus", "num_matches": "296"},
{"account_id": "36564875", "summoner_id": "duoklng1", "num_matches": "11"},
{"account_id": "34389376", "summoner_id": "funnelcakefan69x", "num_matches": "693"},
{"account_id": "35177464", "summoner_id": "Yang Neng Jin", "num_matches": "498"},
{"account_id": "42591150", "summoner_id": "vii is a cutie", "num_matches": "433"},
Error with 201176848 int is success
{"account_id": "40424646", "summoner_id": "lIlIlIIlIlIl", "num_matches": "1022"},
{"account_id": "212650893", "summoner_id": "i need some suga", "num_matches": "33"},
{"account_id": "214201438", "summoner_id": "no hh", "num

i = 34: Waiting for API timer to reset: 100% |##################| Time: 0:02:00


120.6182746887207 seconds elapsed
{"account_id": "200499050", "summoner_id": "why talk", "num_matches": "922"},
{"account_id": "34721678", "summoner_id": "EnGeeeOh", "num_matches": "407"},
{"account_id": "237987641", "summoner_id": "nihonjin", "num_matches": "269"},
{"account_id": "34389376", "summoner_id": "saemi", "num_matches": "693"},
{"account_id": "207797332", "summoner_id": "DesoIatiôn      ", "num_matches": "467"},
Error with 23099233 llIllIllIlllIllI
{"account_id": "36580110", "summoner_id": "OddOJ", "num_matches": "434"},
{"account_id": "36882691", "summoner_id": "Luqun", "num_matches": "408"},
{"account_id": "206113598", "summoner_id": "Terina", "num_matches": "108"},
{"account_id": "229692166", "summoner_id": "7 2 1993", "num_matches": "520"},
{"account_id": "223913076", "summoner_id": "Teamfighting", "num_matches": "24"},
{"account_id": "31757124", "summoner_id": "Hungry Like", "num_matches": "372"},
{"account_id": "207797332", "summoner_id": "300kg XIaoJieJie", "num_match

i = 55: Waiting for API timer to reset: 100% |##################| Time: 0:02:00


120.39510488510132 seconds elapsed
{"account_id": "43592271", "summoner_id": "Useless Whyin", "num_matches": "372"},
{"account_id": "49398962", "summoner_id": "Jue VioIe Grace", "num_matches": "632"},
{"account_id": "211130917", "summoner_id": "Leah cx", "num_matches": "0"},
{"account_id": "206073438", "summoner_id": "147", "num_matches": "402"},
{"account_id": "39734913", "summoner_id": "Jey", "num_matches": "1"},
{"account_id": "37872899", "summoner_id": "TALON NERF PLS", "num_matches": "402"},
{"account_id": "50150916", "summoner_id": "brocco", "num_matches": "263"},
{"account_id": "239623124", "summoner_id": "Game1234", "num_matches": "1407"},
{"account_id": "229692166", "summoner_id": "MrStealYoLP", "num_matches": "520"},
{"account_id": "34376120", "summoner_id": "ltforge", "num_matches": "221"},
Error with 22670400 PePii
{"account_id": "227636156", "summoner_id": "Mìnt", "num_matches": "985"},
{"account_id": "31761738", "summoner_id": "HairySixPack", "num_matches": "271"},
{"acco

...

120.4653742313385 seconds elapsed
{"account_id": "233968031", "summoner_id": "Winter", "num_matches": "930"},
{"account_id": "34486403", "summoner_id": "dog2", "num_matches": "1272"},
{"account_id": "35872324", "summoner_id": "Gweiss", "num_matches": "1678"},
{"account_id": "203021036", "summoner_id": "røvex", "num_matches": "1483"},
{"account_id": "42953504", "summoner_id": "Erry", "num_matches": "604"},
{"account_id": "36541962", "summoner_id": "Shout", "num_matches": "1008"},
{"account_id": "37281196", "summoner_id": "Scrandor", "num_matches": "1624"},


i = 302: Waiting for API timer to reset: 100% |#################| Time: 0:02:00


120.52608680725098 seconds elapsed
{"account_id": "208282883", "summoner_id": "mancloud2", "num_matches": "1572"},
{"account_id": "33064075", "summoner_id": "BukZacH", "num_matches": "1480"},
{"account_id": "44508344", "summoner_id": "Piggy Kitten", "num_matches": "1966"},
{"account_id": "37560515", "summoner_id": "WARMACHINE HYPER", "num_matches": "2445"},
{"account_id": "40424646", "summoner_id": "Nightblue3", "num_matches": "1022"},
{"account_id": "32639237", "summoner_id": "Imaqtpie", "num_matches": "2443"},


i = 308: Waiting for API timer to reset: 100% |#################| Time: 0:02:00


120.54943013191223 seconds elapsed
{"account_id": "32281250", "summoner_id": "Tony Top", "num_matches": "1244"},
{"account_id": "32850898", "summoner_id": "Descraton", "num_matches": "1463"},
{"account_id": "33689527", "summoner_id": "Morïarty", "num_matches": "1460"},
{"account_id": "32746443", "summoner_id": "Shiphtur", "num_matches": "2584"},
{"account_id": "50656739", "summoner_id": "Julien", "num_matches": "1705"},


In [202]:
valid_account_ids[i]

201195334

In [188]:
with open(file_matches, 'w') as fb:
    fb.write(match_ids_as_string)

In [3]:
dat = []
for i, file in enumerate(files):

    with open(file, 'r') as fp:
        match = json.load(fp)
        try:
            cur_dat = process_match(match)
        except:
            print(i,file, 'broken')
            continue
        dat.append([file, np.array(cur_dat)])

0 ./dat/match_2794598133.json broken
1 ./dat/match_2794816148.json broken
2 ./dat/match_2794844157.json broken
3 ./dat/match_2794883638.json broken
4 ./dat/match_2794920723.json broken
5 ./dat/match_2794926208.json broken
6 ./dat/match_2794938006.json broken
7 ./dat/match_2795438660.json broken
8 ./dat/match_2795476074.json broken
9 ./dat/match_2795487487.json broken
10 ./dat/match_2795501212.json broken
11 ./dat/match_2795516056.json broken
12 ./dat/match_2795564932.json broken
13 ./dat/match_2795571169.json broken
14 ./dat/match_2795620883.json broken
15 ./dat/match_2795627955.json broken
16 ./dat/match_2795643784.json broken
17 ./dat/match_2795666997.json broken
18 ./dat/match_2796649534.json broken
19 ./dat/match_2802320642.json broken
20 ./dat/match_2803503351.json broken
21 ./dat/match_2803508301.json broken
22 ./dat/match_2803514209.json broken
23 ./dat/match_2803527717.json broken
24 ./dat/match_2803543989.json broken
25 ./dat/match_2803568212.json broken
26 ./dat/match_2803574

258 ./dat/match_2822955635.json broken
259 ./dat/match_2822960113.json broken
260 ./dat/match_2823011568.json broken
261 ./dat/match_2823027088.json broken
262 ./dat/match_2823037382.json broken
263 ./dat/match_2823051526.json broken
264 ./dat/match_2823090121.json broken
265 ./dat/match_2823094672.json broken
266 ./dat/match_2823163782.json broken
267 ./dat/match_2823182070.json broken
268 ./dat/match_2823216635.json broken
269 ./dat/match_2823236784.json broken
270 ./dat/match_2823237769.json broken
271 ./dat/match_2823275348.json broken
272 ./dat/match_2823290017.json broken
273 ./dat/match_2823296673.json broken
274 ./dat/match_2823333584.json broken
275 ./dat/match_2823433892.json broken
276 ./dat/match_2823496146.json broken
277 ./dat/match_2823521383.json broken
278 ./dat/match_2823537362.json broken
279 ./dat/match_2823571695.json broken
280 ./dat/match_2823580012.json broken
281 ./dat/match_2823612387.json broken
282 ./dat/match_2823620456.json broken
283 ./dat/match_282362695

493 ./dat/match_2833334685.json broken
494 ./dat/match_2833613238.json broken
495 ./dat/match_2833620314.json broken
496 ./dat/match_2833626236.json broken
497 ./dat/match_2833684441.json broken
498 ./dat/match_2833697744.json broken
499 ./dat/match_2833700105.json broken
500 ./dat/match_2833724505.json broken
501 ./dat/match_2833742196.json broken
502 ./dat/match_2833782604.json broken
503 ./dat/match_2833828025.json broken
504 ./dat/match_2833857451.json broken
505 ./dat/match_2833887208.json broken
506 ./dat/match_2833914443.json broken
507 ./dat/match_2833925849.json broken
508 ./dat/match_2833942561.json broken
509 ./dat/match_2833943467.json broken
510 ./dat/match_2833979840.json broken
511 ./dat/match_2833981424.json broken
512 ./dat/match_2834148958.json broken
513 ./dat/match_2834150720.json broken
514 ./dat/match_2834151670.json broken
515 ./dat/match_2834159988.json broken
516 ./dat/match_2834164425.json broken
517 ./dat/match_2834173721.json broken
518 ./dat/match_283419691

In [200]:
match.keys()

dict_keys(['gameId', 'platformId', 'gameCreation', 'gameDuration', 'queueId', 'mapId', 'seasonId', 'gameVersion', 'gameMode', 'gameType', 'teams', 'participants', 'participantIdentities'])